In [ ]:
# default_exp solar

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Solar

> Access data and analysis services that provide access to solar resource data and NREL models using a python API.

In [ ]:
#hide
from nbdev.showdoc import *

# SolarResourceData

In [ ]:
#export
import requests


BASE_URL = "https://developer.nrel.gov"

# TODO - add attributes in docstring, add checks for data inputs
class SolarResourceData:
    """Returns various types of solar data for a location. 
    The service from NREL currently returns data for 
    average Direct Normal Irradiance, average Global Horizontal Irradiance, 
    and average Tilt at Latitude. The data outout format can be json or xml
    """
    
    QUERY_URL = "/api/solar/solar_resource/v1."
    
    def __init__(self, api_key, lat=None, lon=None, address=None, output_format="json"):
        
        self._params = {"api_key": api_key}
        
        # if address is not specified latitude and longitude must be specified
        if not address:
            self._params.update({"lat" : lat, "lon" : lon})
        else:
            self._params.update({"address": address})
        
        self.output_format = output_format
    
    def get(self):
        """Process the get request"""
        resp = requests.get(BASE_URL + self.QUERY_URL + f"{self.output_format}", params=self._params)
        content = resp.json()
        return content
        

An example to get solar resource data from NREL

> In order to use the NREL developer API, you will need an API key from NREL. You can get one [here](https://developer.nrel.gov/signup/). The process is simple and only requires your name and email where you will receive the API key. 

In [ ]:
NREL_API_KEY = "DEMO_KEY"

solar_resource_data = SolarResourceData(api_key=NREL_API_KEY, lat=40, lon=-105, output_format="json")

solar_resource_data.get()

{'version': '1.0.0',
 'warnings': [],
 'errors': [],
 'metadata': {'sources': ['Perez-SUNY/NREL, 2012']},
 'inputs': {'lat': '40', 'lon': '-105'},
 'outputs': {'avg_dni': {'annual': 6.06,
   'monthly': {'jan': 5.0,
    'feb': 5.34,
    'mar': 5.94,
    'apr': 6.11,
    'may': 6.36,
    'jun': 7.43,
    'jul': 7.48,
    'aug': 6.65,
    'sep': 6.81,
    'oct': 5.82,
    'nov': 5.11,
    'dec': 4.67}},
  'avg_ghi': {'annual': 4.81,
   'monthly': {'jan': 2.5,
    'feb': 3.43,
    'mar': 4.69,
    'apr': 5.69,
    'may': 6.6,
    'jun': 7.25,
    'jul': 7.14,
    'aug': 6.24,
    'sep': 5.35,
    'oct': 3.85,
    'nov': 2.75,
    'dec': 2.19}},
  'avg_lat_tilt': {'annual': 5.82,
   'monthly': {'jan': 4.79,
    'feb': 5.4,
    'mar': 6.07,
    'apr': 6.11,
    'may': 6.25,
    'jun': 6.47,
    'jul': 6.58,
    'aug': 6.44,
    'sep': 6.53,
    'oct': 5.71,
    'nov': 4.99,
    'dec': 4.47}}}}

# PVWatts V6

In [ ]:
#export

# TODO - add attributes in docstring, add checks for data inputs
class PVWattsV6:
    """Estimate the energy production of grid-connected photovoltaic (PV) energy systems
    using NREL's PVWatts API based on a few simple inputs.
    """
    
    QUERY_URL = "/api/pvwatts/v6."
    
    def __init__(self,
                 api_key,
                 system_capacity,
                 module_type,
                 losses,
                 array_type,
                 tilt,
                 azimuth,
                 lat=None,
                 lon=None,
                 address=None,
                 file_id=None,
                 dataset="nsrdb",
                 radius=100,
                 timeframe="monthly",
                 dc_ac_ratio=1.2,
                 gcr=0.4,
                 inv_eff=96,
                 output_format="json"):
        
        self._params = {
            "api_key" : api_key,
            "system_capacity" : system_capacity,
            "module_type" : module_type,
            "losses" : losses,
            "array_type" : array_type,
            "tilt" : tilt,
            "azimuth" : azimuth,
        }
        
        # only one of lat/lon, file_id, address needs to be specified
        if not address and not file_id:
            self._params.update({"lat" : lat, "lon" : lon})
        
        if not file_id and not lat and not lon:
            self._params.update({"address" : address})
            
        if not address and not lat and lon:
            self._params.update({"file_id" : file_id})
        
        # if file_id is specified, dataset info is not required
        if not file_id:
            self._params.update({"dataset" : dataset})
        
        self._params.update({
            "radius" : radius,
            "timeframe" : timeframe,
            "dc_ac_ratio" : dc_ac_ratio,
            "gcr" : gcr,
            "inv_eff" : inv_eff
        })
        
        self.output_format = output_format
    
    def get(self):
        """Process the get request"""
        resp = requests.get(BASE_URL + self.QUERY_URL + f"{self.output_format}", params=self._params)
        content = resp.json()
        return content

Example to use PVWattsV6

In [ ]:
pvwatts_v6 = PVWattsV6(api_key=NREL_API_KEY,
                       system_capacity=4,
                       lat=40,
                       lon=-105,
                       azimuth=180,
                       tilt=40,
                       array_type=1,
                       module_type=1,
                       losses=10
                      )

pvwatts_v6.get()

{'inputs': {'system_capacity': '4',
  'module_type': '1',
  'losses': '10',
  'array_type': '1',
  'tilt': '40',
  'azimuth': '180',
  'lat': '40',
  'lon': '-105',
  'dataset': 'nsrdb',
  'radius': '100',
  'timeframe': 'monthly',
  'dc_ac_ratio': '1.2',
  'gcr': '0.4',
  'inv_eff': '96'},
 'errors': [],
 'warnings': [],
 'version': '1.0.2',
 'ssc_info': {'version': 45,
  'build': 'Linux 64 bit GNU/C++ Jul  7 2015 14:24:09'},
 'station_info': {'lat': 40.0099983215332,
  'lon': -105.0199966430664,
  'elev': 1581.839965820312,
  'tz': -7.0,
  'location': 'None',
  'city': '',
  'state': 'Colorado',
  'solar_resource_file': 'W10502N4001.csv',
  'distance': 2029},
 'outputs': {'ac_monthly': [474.4326171875,
   484.3903503417969,
   595.7704467773438,
   592.0599365234375,
   591.2662353515625,
   589.3538208007812,
   583.2352905273438,
   586.4593505859375,
   584.8131713867188,
   561.72314453125,
   486.1260375976562,
   445.6881713867188],
  'poa_monthly': [141.4809417724609,
   145.5

# SolarDatasetQuery

In [ ]:
#export

# TODO - add attributes in docstring, add checks for data inputs
class SolarDatasetQuery:
    
    """Returns information on the closest climate data for a location.
    """
    
    QUERY_URL = "/api/solar/data_query/v1."
    
    def __init__(self,
                 api_key,
                 lat=None,
                 lon=None,
                 address=None,
                 radius=100,
                 return_all_stations=False,
                 output_format="json"
                ):
        
        self._params = {
            "api_key" : api_key,
            "radius" : radius,
            "all" : 0 if return_all_stations is False else 1,
        }
        
        # if address is not specified latitude and longitude must be specified
        if not address:
            self._params.update({"lat" : lat, "lon" : lon})
        else:
            self._params.update({"address": address})
        
        self.output_format = output_format
        
    def get(self):
        """Process the get request"""
        resp = requests.get(BASE_URL + self.QUERY_URL + f"{self.output_format}", params=self._params)
        content = resp.json()
        return content

Information on climate data for the location closest to the specified one.

In [ ]:
solar_dataset_query = SolarDatasetQuery(api_key=NREL_API_KEY,
                                        lat=40,
                                        lon=-105,
                                       )

solar_dataset_query.get()

{'version': '1.3.2',
 'warnings': ['No intl data found for lat=40.0 lon=-105.0'],
 'errors': [],
 'metadata': {},
 'inputs': {'lat': '40', 'lon': '-105', 'radius': '100', 'all': '0'},
 'outputs': {'tmy2': {'id': '0-94018',
   'city': 'BOULDER',
   'state': 'COLORADO',
   'timezone': -7,
   'lat': 40.13,
   'lon': -105.24,
   'elevation': 1689,
   'distance': 24977},
  'tmy3': {'id': '1-724699',
   'city': 'BROOMFIELD/JEFFCO [BOULDER - SURFRAD]',
   'state': 'COLORADO',
   'timezone': -7,
   'lat': 40.13,
   'lon': -105.24,
   'elevation': 1689,
   'distance': 24977},
  'intl': None,
  'nsrdb': {'id': '3-W105N040-W10502N4001',
   'city': '',
   'state': 'Colorado',
   'country': None,
   'lat': 40.01,
   'lon': -105.02,
   'distance': 2029,
   'timezone': -7,
   'elevation': 1582,
   'resolution': 4}}}

Get information about all the stations within the specified radius

In [ ]:
solar_dataset_query = SolarDatasetQuery(api_key=NREL_API_KEY,
                                        lat=40,
                                        lon=-105,
                                        radius=100,
                                        return_all_stations=True
                                       )

solar_dataset_query.get()

{'version': '1.3.2',
 'warnings': ['No intl data found for lat=40.0 lon=-105.0'],
 'errors': [],
 'metadata': {},
 'inputs': {'lat': '40', 'lon': '-105', 'radius': '100', 'all': '1'},
 'outputs': {'tmy2': {'id': '0-94018',
   'city': 'BOULDER',
   'state': 'COLORADO',
   'timezone': -7,
   'lat': 40.13,
   'lon': -105.24,
   'elevation': 1689,
   'distance': 24977},
  'tmy3': {'id': '1-724699',
   'city': 'BROOMFIELD/JEFFCO [BOULDER - SURFRAD]',
   'state': 'COLORADO',
   'timezone': -7,
   'lat': 40.13,
   'lon': -105.24,
   'elevation': 1689,
   'distance': 24977},
  'intl': None,
  'nsrdb': {'id': '3-W105N040-W10502N4001',
   'city': '',
   'state': 'Colorado',
   'country': None,
   'lat': 40.01,
   'lon': -105.02,
   'distance': 2029,
   'timezone': -7,
   'elevation': 1582,
   'resolution': 4},
  'all_stations': [{'id': '3-W105N040-W10502N4001',
    'city': '',
    'state': 'Colorado',
    'country': None,
    'lat': 40.01,
    'lon': -105.02,
    'distance': 2029,
    'timezone'